In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Paths and stuff
import os
import sys

sys.path.append('/home/shreyas/pySICOPOLIS/src')
from pySICOPOLIS import *

In [2]:
ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_init = xr.open_dataset('/home/shreyas/update_to_develop_sicopolis/sicopolis_spinups/sico_out/grl40_bm5_paleo17a_CT4_BH0_13point5CS_spinup_0ka/grl40_bm5_paleo17a_CT4_BH0_13point5CS_spinup_0ka0006.nc')
ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_final = xr.open_dataset('/home/shreyas/update_to_develop_sicopolis/sicopolis_spinups/sico_out/grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp/grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp0006.nc')
H_data_40 = xr.open_dataset("/scratch2/shreyas/GrIS_paleo_data/bm5_data_40kms.nc")
age_data_40 = xr.open_dataset("/scratch2/shreyas/GrIS_paleo_data/age_data_40kms.nc")

H_data = H_data_40["H"].data
H_uncert_data = H_data_40["H_uncert"].data
age_c_data = age_data_40["age_c"].data
age_c_uncert_real_data = age_data_40["age_c_uncert_real"].data

mask_H = H_data >= 0.0
mask_H = mask_H.astype(float)
mask_age_c = np.zeros(age_c_data.shape)

for kc in range(age_c_data.shape[0]):
    for j in range(age_c_data.shape[1]):
        for i in range(age_c_data.shape[2]):
            if age_c_uncert_real_data[kc, j, i] > 0 and age_c_data[kc, j, i] >= 0 and age_c_data[kc, j, i] <= 60000 and H_data[j, i] >= 2000.0:
                mask_age_c[kc, j, i] = 1.0

In [3]:
sicopolis_dir = '/home/shreyas/update_to_develop_sicopolis/sicopolis_tuneAll'
simulation = 'grl40_bm5_paleo17a_CT4_BH0_AC_BM5_m11ka_pkp'
dict_sico_out_folder_prefixes = {"nodiff": "N",
                                 "tlm": "FORWARD",
                                 "adj": "ADJOINT",
                                 "tlm_action": "FORWARDHESSACTION",
                                 "adj_action": "ADJHESSACTION"}
dict_ad_exec_cmds_suffixes = {"nodiff": "nodiff",
                              "tlm": "forward",
                              "adj": "adjoint",
                              "tlm_action": "forwardhessaction",
                              "adj_action": "adjointhessaction"}
dict_ad_log_file_suffixes = {"nodiff": "nodiff",
                             "tlm": "tlm",
                             "adj": "adj",
                             "tlm_action": "tlm_hessaction",
                             "adj_action": "adj_hessaction"}
dict_ad_nc_suffixes = {"nodiff": "nodiff",
                       "tlm": "tlm",
                       "adj": "adj",
                       "tlm_action": "tlm_hessaction",
                       "adj_action": "adj_hessaction"}

KCMAX = 80
exp_sigma_level = dataCleaner.exp_sigma_level(zeta = np.arange(0,1+1./KCMAX,1./KCMAX),
                                              exponent = 2.0)
xModel40       = np.arange(-72.,97.,4.0)*10
yModel40       = np.arange(-345.,-56.,4.0)*10
time_ad = np.arange(12, dtype=float)
IMAX = xModel40.shape[0]-1
JMAX = yModel40.shape[0]-1

data = ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_init['q_geo'].data
log_q_geo = np.where(data > 0, np.log10(data), -100)
data = ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_init['H'].data
log_H = np.where(data > 0, np.log10(data), -100)

log_c_slide_init = np.log10(13.5)*np.ones((JMAX+1, IMAX+1))
log_gamma_s = np.log10(0.070458)*np.ones((JMAX+1, IMAX+1))
log_s_stat = np.log10(5.0)
log_beta1 = np.log10(2.73)
log_beta2 = np.log10(7.28)
log_Pmax = np.log10(0.6)
log_mu = np.log10(9.7155)
log_c_dis_da = np.log10(15659.0)

temp_c = np.zeros((KCMAX+1, JMAX+1, IMAX+1), dtype=float)
data = ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_init['age_c'].data
log_age_c = np.where(data > 0, np.log10(data), -100)

dict_og_state_fields_vals = {"xx_c_slide_init": log_c_slide_init,
                    "xx_q_geo": log_q_geo,
                    "xx_H": log_H,
                    "xx_gamma_s": log_gamma_s,
                    "xx_s_stat": log_s_stat,
                    "xx_beta1": log_beta1,
                    "xx_beta2": log_beta2,
                    "xx_Pmax": log_Pmax,
                    "xx_mu": log_mu,
                    "xx_c_dis_da": log_c_dis_da,
                    "xx_temp_c": temp_c,
                    "xx_age_c": log_age_c}

dict_state_fields_num_dims = {"xx_c_slide_init": "2D",
                        "xx_q_geo": "2D",
                        "xx_H": "2D",
                        "xx_gamma_s": "2D",
                        "xx_s_stat": "2D",
                        "xx_beta1": "2D",
                        "xx_beta2": "2D",
                        "xx_Pmax": "2D",
                        "xx_mu": "2D",
                        "xx_c_dis_da": "2D",
                        "xx_temp_c": "3D",
                        "xx_age_c": "3D"}

dict_state_coords = {"time_ad": time_ad,
               "zeta_c": exp_sigma_level,
               "y": yModel40,
               "x": xModel40}

dict_state_attrs_type = {"xx_c_slide_init": "nodiff",
                   "xx_q_geo": "nodiff",
                   "xx_H": "nodiff",
                   "xx_gamma_s": "nodiff",
                   "xx_s_stat": "nodiff",
                   "xx_beta1": "nodiff",
                   "xx_beta2": "nodiff",
                   "xx_Pmax": "nodiff",
                   "xx_mu": "nodiff",
                   "xx_c_dis_da": "nodiff",
                   "xx_temp_c": "nodiff",
                   "xx_age_c": "nodiff"}

dict_state_fields_or_scalars = {"xx_c_slide_init": "field",
                          "xx_q_geo": "field",
                          "xx_H": "field",
                          "xx_gamma_s": "field",
                          "xx_s_stat": "scalar",
                          "xx_beta1": "scalar",
                          "xx_beta2": "scalar",
                          "xx_Pmax": "scalar",
                          "xx_mu": "scalar",
                          "xx_c_dis_da": "scalar",
                          "xx_temp_c": "field",
                          "xx_age_c": "field"}

dict_masks_observables = {"H": mask_H*H_uncert_data**(-2)}

/tmp/ipykernel_425939/294431682.py:36: RuntimeWarning: divide by zero encountered in log10
  log_H = np.where(data > 0, np.log10(data), -100)
/tmp/ipykernel_425939/294431682.py:49: RuntimeWarning: divide by zero encountered in log10
  log_age_c = np.where(data > 0, np.log10(data), -100)


In [4]:
DA = optim_new.DataAssimilation(sicopolis_dir, simulation,
                                dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                                dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                                dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars, dict_masks_observables,
                                ["xx_s_stat", "xx_c_slide_init"])

ds_inp_nodiff = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "nodiff")
ds_inp_adj = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                 dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                 "adj")

fc = DA.eval_cost()

ds_x = DA.create_ad_tlm_action_input_nc(bool_randomize = True)
ds_x_tlm_only = DA.subset_of_ds(ds_x, "type", "tlm")
    
ds_Ax = DA.eval_tlm_action()
ds_noise_cov_inv_Ax = DA.eval_noise_cov_inv_action(ds_Ax)
ds_H_misfit_x = DA.eval_misfit_hessian_action()

DA.l2_inner_product([ds_x_tlm_only, ds_H_misfit_x], ["tlm", "adj"]), DA.l2_inner_product([ds_Ax, ds_noise_cov_inv_Ax], ["tlmhessaction", "adjhessaction"])

(24865478.81340736, 24865478.81340737)

In [5]:
DA = optim_new.DataAssimilation(sicopolis_dir, simulation,
                                dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                                dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                                dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars, dict_masks_observables,
                                ["xx_s_stat", "xx_c_slide_init"])

ds_inp_nodiff = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "nodiff")
ds_inp_adj = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                 dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                 "adj")

fc = DA.eval_cost()

dict_tlm_action_only_fields_vals = {}
for var in ds_x_tlm_only:
    if DA.dict_tlm_action_fields_or_scalars[var] == "scalar":
        dict_tlm_action_only_fields_vals[var] = ds_x_tlm_only[var].data[0].copy()
    else:
        dict_tlm_action_only_fields_vals[var] = ds_x_tlm_only[var].data.copy()
ds_x = DA.create_ad_tlm_action_input_nc(dict_tlm_action_only_fields_vals)
    
ds_Ax = DA.eval_tlm_action()
ds_noise_cov_inv_Ax = DA.eval_noise_cov_inv_action(ds_Ax)
ds_H_misfit_x = DA.eval_misfit_hessian_action()

DA.l2_inner_product([ds_x_tlm_only, ds_H_misfit_x], ["tlm", "adj"]), DA.l2_inner_product([ds_Ax, ds_noise_cov_inv_Ax], ["tlmhessaction", "adjhessaction"])

(24865478.81340736, 24865478.81340737)

In [6]:
DA = optim_new.DataAssimilation(sicopolis_dir, simulation,
                                dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                                dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                                dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars, dict_masks_observables,
                                ["xx_c_slide_init"])

ds_inp_nodiff = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "nodiff")
ds_inp_adj = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                 dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                 "adj")

DA.gradient_descent(10, init_alpha = 1.e-6)

-------------------------------------
iter 0, fc = 2459227.2817783635
-------------------------------------
Step size alpha = 1e-06
-------------------------------------
iter 1, fc = 917817.9587995579
-------------------------------------
Step size alpha = 1e-06
-------------------------------------
iter 2, fc = 687456.6754333116
-------------------------------------
Step size alpha = 1e-06
-------------------------------------
iter 3, fc = 569887.9814043919
-------------------------------------
Step size alpha = 1e-06
-------------------------------------
iter 4, fc = 496146.63335662166
-------------------------------------
Step size alpha = 1e-06
-------------------------------------
iter 5, fc = 439370.18971046817
-------------------------------------
Step size alpha = 1e-06
-------------------------------------
iter 6, fc = 393286.71631031885
-------------------------------------
Step size alpha = 1e-06
-------------------------------------
iter 7, fc = 354867.7681635089
----------

<xarray.Dataset> Size: 4MB
Dimensions:          (y: 73, x: 43, scalar: 1, zeta_c: 81)
Coordinates:
  * x                (x) float64 344B -7.2e+05 -6.8e+05 ... 9.2e+05 9.6e+05
  * y                (y) float64 584B -3.45e+06 -3.41e+06 ... -6.1e+05 -5.7e+05
Dimensions without coordinates: scalar, zeta_c
Data variables:
    xx_c_slide_init  (y, x) float64 25kB ...
    xx_q_geo         (y, x) float64 25kB -1.286 -1.28 -1.275 ... -1.145 -1.146
    xx_H             (y, x) float64 25kB -100.0 -100.0 -100.0 ... -100.0 -100.0
    xx_gamma_s       (y, x) float64 25kB -1.152 -1.152 -1.152 ... -1.152 -1.152
    xx_s_stat        (scalar) float64 8B 0.8417
    xx_beta1         (scalar) float64 8B 0.4743
    xx_beta2         (scalar) float64 8B 0.9002
    xx_Pmax          (scalar) float64 8B -0.2737
    xx_mu            (scalar) float64 8B 0.9875
    xx_c_dis_da      (scalar) float64 8B 4.487
    xx_temp_c        (zeta_c, y, x) float64 2MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    xx_age_c         (zeta_c, y, x) float64 2MB -100.0 -100.0 ... -100.0 -100.0
Attributes:
    history:  2024-12-26 21:23:34 -06:00 - Data produced

In [7]:
DA = optim_new.DataAssimilation(sicopolis_dir, simulation,
                                dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                                dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                                dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars, dict_masks_observables,
                                ["xx_c_slide_init"])

ds_inp_nodiff = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "nodiff")
ds_inp_adj = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                 dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                 "adj")

DA.inexact_gn_hessian_cg(MAX_ITERS=10, init_alpha_gd = 1.e-6)

-------------------------------------
Initial fc = 2459227.2817783635
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
CG iter 4
conjugate_gradient: Convergence.
Angle between p and g in degrees:  112.53986992956222
Step size alpha = 1.0
-------------------------------------
Outer iter 1, fc = 2441170.698705728
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
conjugate_gradient: Convergence.
Angle between p and g in degrees:  165.99760659458153
Step size alpha = 1.0
-------------------------------------
Outer iter 2, fc = 2320530.9263110277
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
conjugate_gradient: Convergence.
Angle between p and g in degrees:  165.93643829497142
Step size alpha = 1.0
-------------------------------------
Outer iter 3, fc = 1932224.698198685
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
conjugate_gradient: Convergence.
Angle between p and g in degrees:  161.9358272179365
Step size

<xarray.Dataset> Size: 4MB
Dimensions:          (y: 73, x: 43, scalar: 1, zeta_c: 81)
Coordinates:
  * x                (x) float64 344B -7.2e+05 -6.8e+05 ... 9.2e+05 9.6e+05
  * y                (y) float64 584B -3.45e+06 -3.41e+06 ... -6.1e+05 -5.7e+05
Dimensions without coordinates: scalar, zeta_c
Data variables:
    xx_c_slide_init  (y, x) float64 25kB ...
    xx_q_geo         (y, x) float64 25kB ...
    xx_H             (y, x) float64 25kB ...
    xx_gamma_s       (y, x) float64 25kB ...
    xx_s_stat        (scalar) float64 8B 1.98
    xx_beta1         (scalar) float64 8B 0.7829
    xx_beta2         (scalar) float64 8B 1.197
    xx_Pmax          (scalar) float64 8B -0.6918
    xx_mu            (scalar) float64 8B 0.9875
    xx_c_dis_da      (scalar) float64 8B 6.857
    xx_temp_c        (zeta_c, y, x) float64 2MB ...
    xx_age_c         (zeta_c, y, x) float64 2MB ...
Attributes:
    history:  2024-12-26 21:25:58 -06:00 - Data produced

In [8]:
DA = optim_new.DataAssimilation(sicopolis_dir, simulation,
                                dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                                dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                                dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars, dict_masks_observables,
                                ["xx_c_slide_init"])

ds_inp_nodiff = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "nodiff")
ds_inp_adj = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                 dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                 "adj")

DA.inexact_gn_hessian_cg(MAX_ITERS=10, init_alpha_gd = 1.e-6, cg_tolerance_type = "linear")

-------------------------------------
Initial fc = 2459227.2817783635
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
CG iter 4
conjugate_gradient: Convergence.
Angle between p and g in degrees:  112.53986992956222
Step size alpha = 1.0
-------------------------------------
Outer iter 1, fc = 2441170.698705728
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
conjugate_gradient: Convergence.
Angle between p and g in degrees:  165.99760659458153
Step size alpha = 1.0
-------------------------------------
Outer iter 2, fc = 2320530.9263110277
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
conjugate_gradient: Convergence.
Angle between p and g in degrees:  165.93643829497142
Step size alpha = 1.0
-------------------------------------
Outer iter 3, fc = 1932224.698198685
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
conjugate_gradient: Convergence.
Angle between p and g in degrees:  161.9358272179365
Step size

<xarray.Dataset> Size: 4MB
Dimensions:          (y: 73, x: 43, scalar: 1, zeta_c: 81)
Coordinates:
  * x                (x) float64 344B -7.2e+05 -6.8e+05 ... 9.2e+05 9.6e+05
  * y                (y) float64 584B -3.45e+06 -3.41e+06 ... -6.1e+05 -5.7e+05
Dimensions without coordinates: scalar, zeta_c
Data variables:
    xx_c_slide_init  (y, x) float64 25kB ...
    xx_q_geo         (y, x) float64 25kB ...
    xx_H             (y, x) float64 25kB ...
    xx_gamma_s       (y, x) float64 25kB ...
    xx_s_stat        (scalar) float64 8B 1.98
    xx_beta1         (scalar) float64 8B 0.7829
    xx_beta2         (scalar) float64 8B 1.197
    xx_Pmax          (scalar) float64 8B -0.6918
    xx_mu            (scalar) float64 8B 0.9875
    xx_c_dis_da      (scalar) float64 8B 6.857
    xx_temp_c        (zeta_c, y, x) float64 2MB ...
    xx_age_c         (zeta_c, y, x) float64 2MB ...
Attributes:
    history:  2024-12-26 21:28:18 -06:00 - Data produced